<a href="https://colab.research.google.com/github/kishon45229/Customer-Churn-Prediction-in-Telecom-Industry/blob/main/Data_preprocessing_part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Customer Churn Prediction in Telecom Industry

This repository focuses on predicting customer churn in the telecom industry. The data preprocessing section is key to ensuring the accuracy of the predictive model. It involves handling missing values, encoding categorical variables into numerical formats, and performing feature engineering to create and modify features that enhance model performance. Additionally, attribute subset selection techniques, such as Recursive Feature Elimination (RFE), are applied to identify the most relevant features. These preprocessing steps are essential for building a reliable churn prediction model, helping telecom companies proactively retain their customers.

ITBIN-2110-0031

TF.FATHIMA

# Data preprocessing part 2

# Import Necessary Libraries

In [12]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

# Add Dataset
I am going to continue from previous section data preprocessing part 1. therefore I read the csv file that completed the data preprocessing part.

In [13]:
df = pd.read_csv('/content/drive/MyDrive/Section 2 finished dataset.csv')

In [15]:
df.head()


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,tenure_bin,MonthlyCharges_bin,Cluster,PCA1,PCA2
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,Yes,Electronic check,29.85,29.85,No,0-12,21-40,0,-2.186195,-0.654456
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,No,Mailed check,56.95,1889.50,No,25-36,41-60,2,-0.115181,0.583276
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,Yes,Mailed check,53.85,108.15,Yes,0-12,41-60,2,-1.311383,1.343579
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,No,Bank transfer (automatic),42.30,1840.75,No,37-48,41-60,0,-0.570548,-1.533997
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,Yes,Electronic check,70.70,151.65,Yes,0-12,61-80,2,-0.992743,1.557438


# Reduction
Cube Aggregation

Cube aggregation is a technique used in data warehousing and multidimensional databases to summarize and aggregate data along various dimensions. It involves creating a data cube, which is a multi-dimensional array of values, and then performing operations to summarize the data at different levels of granularity.

In the context of our dataset, cube aggregation can help in summarizing and understanding customer data by aggregating values along various dimensions such as tenure, MonthlyCharges, and TotalCharges.

We did cube aggregation using groupby() on the dataset by aggregating the MonthlyCharges and TotalCharges along the tenure_bin and Contract dimensions.

In [16]:
cube_aggregation = df.groupby(['tenure_bin', 'Contract'])[['MonthlyCharges', 'TotalCharges']].agg(['mean', 'sum', 'count'])

Next, we reset index to flatten the DataFrame for better readability.

In [17]:
cube_aggregation = cube_aggregation.reset_index()
cube_aggregation

tenure_bin        Contract MonthlyCharges                  TotalCharges  \
                                        mean        sum count         mean   
0        0-12  Month-to-month      58.217904  116086.50  1994   276.693430   
1        0-12        One year      35.928455    4419.20   123   303.171545   
2        0-12        Two year      28.766379    1668.45    58   217.846552   
3       13-24  Month-to-month      69.309566   51081.15   737  1257.884193   
4       13-24        One year      44.878680    8841.10   197   863.225381   
5       13-24        Two year      32.306667    2907.60    90   624.129444   
6       25-36  Month-to-month      74.326235   36122.55   486  2238.959979   
7       25-36        One year      58.098800   14524.70   250  1777.244400   
8       25-36        Two year      40.745313    3911.55    96  1285.418229   
9       37-48  Month-to-month      78.422468   24781.50   316  3308.503639   
10      37-48        One year      65.554478   17568.60   268  2803.992537   
11      37-48        Two year      45.979775    8184.40   178  2008.861798   
12      49-60  Month-to-month      82.679274   19346.95   234  4479.145513   
13      49-60        One year      75.128037   24116.10   321  4087.001558   
14      49-60        Two year      55.000722   15235.20   277  3038.262635   
15      61-72  Month-to-month      91.439815    9875.50   108  5975.300000   
16      61-72        One year      84.112460   26327.20   313  5597.442173   
17      61-72        Two year      71.666075   70662.75   986  4961.329260   

                      
           sum count  
0    551726.70  1994  
1     37290.10   123  
2     12635.10    58  
3    927060.65   737  
4    170055.40   197  
5     56171.65    90  
6   1088134.55   486  
7    444311.10   250  
8    123400.15    96  
9   1045487.15   316  
10   751470.00   268  
11   357577.40   178  
12  1048120.05   234  
13  1311927.50   321  
14   841598.75   277  
15   645332.40   108  
16  1751999.40   313  
17  4891870.65   986

The output is a DataFrame. It is showing a summarized statistics for MonthlyCharges and TotalCharges for each combination of tenure_bin and Contract. This can help in understanding patterns such as:

The mean values indicate the average monthly and total charges for customers in each tenure bin and contract type.
The sum values give the total revenue generated from customers in each category.
The count values show how many customers fall into each category.

# Attribute Subset Selection/Feature Selection
It is a process in machine learning and data preprocessing where you select a subset of relevant features (attributes) for building your model. The goal is to improve the model's performance by removing irrelevant, redundant, or noisy features.

We need to define feature(X) and target(y). All other columns as features expect target column. We choosed target as Churn column because in this dataset, churn prediction is typically a key business objective. Churn refers to the phenomenon where customers stop using the company's services, and predicting churn can help the company identify customers at risk of leaving. This allows the company to take proactive measures to retain those customers.

In [18]:
df['Churn'] = df['Churn'].apply(lambda x: 1 if x == 'Yes' else 0)

X = df.drop(['Churn'], axis=1)
y = df['Churn']

The below code will convert categorical variables to numerical(if any)

In [19]:
X = pd.get_dummies(X, drop_first=True)

Now we applied feature selection method to identify the most relevant features. In the below code we wrote code to select top 20 features.

In [28]:
selector = SelectKBest(score_func=f_classif, k=20)
X_selected = selector.fit_transform(X, y)

NameError: name 'SelectKBest' is not defined

# Normalization